# Image Dehazing using GMAN Net
Github Link: https://github.com/sanchitvj/Image-Dehazing-using-GMAN-net

## Main and Defined Functions:

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import random
from PIL import Image
import time
import datetime
import cv2
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import imutils
from imutils.video import VideoStream
%load_ext autotime


t_net = tf.keras.models.load_model(r'C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\Image-Dehazing-using-GMAN-net\saved_model', compile = False)

time: 2.36 s (started: 2022-12-15 19:16:52 +05:30)


## Single Image Dehazing Code:

In [5]:
test_img_path = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images"

test_img = glob.glob(test_img_path + '/*.jpg')
# random.shuffle(test_img)

for img in test_img:
  img = Image.open(img)
  w, h = img.size
  img = img.resize((548, 412))
  img = np.asarray(img) / 255.0
  img = img.reshape((1,) + img.shape)

  dehaze = t_net(img, training = False)

  img = tf.reshape(dehaze,[412,548,3])

  RGB_img = cv2.cvtColor(img.numpy(), cv2.COLOR_BGR2RGB)
  RGB_img = cv2.resize(RGB_img, (w,h))
#   print(RGB_img.shape)

#   cv2.imshow('img',RGB_img)
#   Saving the Image:
  cv2.imwrite(r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\img.png',RGB_img*255)

  cv2.waitKey(0) & 0xFF == ord('q')
  cv2.destroyAllWindows()

time: 1.55 s (started: 2022-12-15 18:55:14 +05:30)


### OR

In [10]:
# test_img_path = r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images'

# test_img = glob.glob(test_img_path + '/*.jpg')
# random.shuffle(test_img)

# for img in test_img:
#   img = tf.io.read_file(img)
#   img = tf.io.decode_jpeg(img, channels = 3)
#   h, w, a = np.array(img).shape
#   img = tf.image.resize(img, size = (412, 548), antialias = True)
#   img = img / 255.0
#   img = tf.expand_dims(img, axis = 0)

#   dehaze = t_net(img, training = False)

#   img = tf.reshape(dehaze,[412,548,3])

#   RGB_img = cv2.cvtColor(img.numpy(), cv2.COLOR_BGR2RGB)
#   RGB_img = cv2.resize(RGB_img, (w,h))
# #   print(RGB_img.shape)

#   cv2.imshow('img',RGB_img)
# #   Saving the Image:
# #   cv2.imwrite(r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\img.png',RGB_img)

#   cv2.waitKey(0) & 0xFF == ord('q')
#   cv2.destroyAllWindows()

## Video Dehazing Code:

In [ ]:
inputVideoPath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\Vdo_3.mp4"
vidObj = cv2.VideoCapture(inputVideoPath)

# Loop for frames:
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
frame_width = vidObj.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = vidObj.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = vidObj.get(cv2.CAP_PROP_FPS)
frame_nos = vidObj.get(cv2.CAP_PROP_FRAME_COUNT)
# save = cv2.VideoWriter(r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\output_small.mp4',fourcc, fps, (640, 480))
save = cv2.VideoWriter(r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\output_small3.mp4',fourcc, fps, (int(frame_width), int(frame_height)))

###
for frame_idx in range(int(frame_nos+1)):
  ret, I = vidObj.read()

  if ret:
    I = cv2.resize(I, (548, 412))
    x = I / 255.0
    x = x.reshape((1,) + x.shape)
    
    dehaze = t_net(x, training = False)

    x = tf.reshape(dehaze,[412,548,3])
    x = cv2.cvtColor(x.numpy(), cv2.COLOR_BGR2RGB)
    x = x * 255
    x = (np.rint(x)).astype(np.uint8)
    x = cv2.resize(x, (int(frame_width), int(frame_height)))
    save.write(x)
    
  # Breaking Loop:
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

vidObj.release()
save.release()
cv2.destroyAllWindows()

print('Execution Completed')

## Real-Time Dehazing Code:

In [3]:
if __name__ == "__main__":

    vidObj_vs = VideoStream(0).start()
    
    time.sleep(1.0)
    start_time = time.time()
    
    # Defining FPS:
    FPS_Start_Time = 0
    FPS = 0
    
    while True:

        I = vidObj_vs.read()
        # FPS:
        h, w, a = I.shape
        FPS_End_Time = time.time()
        FPS = 1/(FPS_End_Time - FPS_Start_Time)
        FPS_Start_Time = FPS_End_Time
        FPS_Text = "FPS: {:.2f}".format(FPS)
        cv2.putText(I, FPS_Text, (5,20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,0,0), 1)
        
        cv2.imshow('Input Sequence',I)
        
        I = cv2.resize(I, (548, 412))
        x = I / 255.0
        x = x.reshape((1,) + x.shape)
        
        dehaze = t_net(x, training = False)
        
        x = tf.reshape(dehaze,[412,548,3])
#         x = cv2.cvtColor(x.numpy(), cv2.COLOR_BGR2RGB)
        x = x * 255
        x = (np.rint(x)).astype(np.uint8)
        x = cv2.resize(x, (w,h))
        cv2.imshow('Output Sequence',x)

        # Breaking Loop:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vidObj_vs.stream.release()
    # save.release()
    cv2.destroyAllWindows()